# Week 9, Day 6: Generative AI Hackathon Challenge

## Challenge Overview
Build an end-to-end generative AI solution using concepts learned throughout Week 9:
- Generative Models
- GANs and VAEs
- Diffusion Models
- Language Models
- Advanced NLP

## Problem: Multi-Modal Generation System
Create a system that can generate multiple types of content:
1. Text Generation
2. Image Generation
3. Text-to-Image Generation
4. Style Transfer

In [ ]:
# Import required libraries
import numpy as np
import tensorflow as tf
import torch
from transformers import pipeline
from PIL import Image
import matplotlib.pyplot as plt

## Part 1: Text Generation System

In [ ]:
class TextGenerator:
    def __init__(self):
        # Initialize text generation pipeline
        self.generator = pipeline('text-generation', model='gpt2')
    
    def generate(self, prompt, max_length=100, num_samples=1, temperature=0.7):
        """Generate text based on prompt"""
        outputs = self.generator(
            prompt,
            max_length=max_length,
            num_return_sequences=num_samples,
            temperature=temperature
        )
        return [out['generated_text'] for out in outputs]
    
    def batch_generate(self, prompts, **kwargs):
        """Generate text for multiple prompts"""
        all_outputs = []
        for prompt in prompts:
            outputs = self.generate(prompt, **kwargs)
            all_outputs.extend(outputs)
        return all_outputs

## Part 2: Image Generation System

In [ ]:
class ImageGenerator:
    def __init__(self):
        # Initialize image generation model
        self.model = self.build_generator()
    
    def build_generator(self):
        model = tf.keras.Sequential([
            # Latent space input
            tf.keras.layers.Input(shape=(100,)),
            
            # Dense layer and reshape
            tf.keras.layers.Dense(8*8*256),
            tf.keras.layers.Reshape((8, 8, 256)),
            
            # Upsampling blocks
            tf.keras.layers.Conv2DTranspose(128, 4, strides=2, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            
            tf.keras.layers.Conv2DTranspose(64, 4, strides=2, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            
            tf.keras.layers.Conv2DTranspose(32, 4, strides=2, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            
            # Output layer
            tf.keras.layers.Conv2D(3, 4, padding='same', activation='tanh')
        ])
        return model
    
    def generate(self, num_images=1):
        """Generate random images"""
        noise = tf.random.normal([num_images, 100])
        generated_images = self.model(noise, training=False)
        return generated_images
    
    def generate_and_plot(self, num_images=4):
        """Generate and display images"""
        images = self.generate(num_images)
        
        plt.figure(figsize=(10, 10))
        for i in range(num_images):
            plt.subplot(2, 2, i+1)
            plt.imshow(images[i] * 0.5 + 0.5)
            plt.axis('off')
        plt.show()

## Part 3: Text-to-Image System

In [ ]:
class TextToImageGenerator:
    def __init__(self):
        # Initialize text encoder and image decoder
        self.text_encoder = self.build_text_encoder()
        self.image_decoder = self.build_image_decoder()
    
    def build_text_encoder(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(100,)),  # Tokenized text
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(1024)
        ])
        return model
    
    def build_image_decoder(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(1024,)),
            tf.keras.layers.Dense(8*8*256),
            tf.keras.layers.Reshape((8, 8, 256)),
            
            tf.keras.layers.Conv2DTranspose(128, 4, strides=2, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            
            tf.keras.layers.Conv2DTranspose(64, 4, strides=2, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            
            tf.keras.layers.Conv2DTranspose(32, 4, strides=2, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            
            tf.keras.layers.Conv2D(3, 4, padding='same', activation='tanh')
        ])
        return model
    
    def generate(self, text_embedding):
        """Generate image from text embedding"""
        encoded_text = self.text_encoder(text_embedding)
        generated_image = self.image_decoder(encoded_text)
        return generated_image

## Part 4: Style Transfer System

In [ ]:
class StyleTransfer:
    def __init__(self):
        # Initialize style transfer model
        self.model = self.build_model()
    
    def build_model(self):
        model = tf.keras.Sequential([
            # Content encoder
            tf.keras.layers.Input(shape=(256, 256, 3)),
            tf.keras.layers.Conv2D(32, 3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            
            # Style encoder
            tf.keras.layers.Conv2D(64, 3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            
            # Transformer
            tf.keras.layers.Conv2D(128, 3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            
            # Decoder
            tf.keras.layers.Conv2D(64, 3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            
            tf.keras.layers.Conv2D(32, 3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            
            tf.keras.layers.Conv2D(3, 3, padding='same', activation='tanh')
        ])
        return model
    
    def transfer_style(self, content_image, style_image):
        """Apply style transfer"""
        # Preprocess images
        content = tf.image.resize(content_image, [256, 256])
        style = tf.image.resize(style_image, [256, 256])
        
        # Combine and process
        combined = tf.concat([content, style], axis=-1)
        result = self.model(combined)
        
        return result

## Evaluation Criteria

Your solution will be evaluated based on:

1. Text Generation (25%)
   - Text quality
   - Coherence
   - Diversity

2. Image Generation (25%)
   - Image quality
   - Diversity
   - Resolution

3. Text-to-Image (25%)
   - Alignment
   - Quality
   - Creativity

4. Style Transfer (25%)
   - Style fidelity
   - Content preservation
   - Transfer quality

## Submission Guidelines
1. Complete all tasks in this notebook
2. Document your approach and decisions
3. Include visualizations and examples
4. Provide suggestions for improvement